In this notebook, we compute the maximal (resp. minimal) volume ellispoids and polynomial sublevel sets contained (containing) the square with vertices $(\pm 1, \pm 1)$. We start by defining the square with [Polyhedra](https://github.com/JuliaPolyhedra/Polyhedra.jl).

In [ ]:
using Polyhedra
h = HalfSpace([1, 0], 1.0) ∩ HalfSpace([-1, 0], 1) ∩ HalfSpace([0, 1], 1) ∩ HalfSpace([0, -1], 1)
p = polyhedron(h);

We need to pick an SDP solver, see [here](http://www.juliaopt.org/JuMP.jl/dev/installation/#Getting-Solvers-1) for a list of available ones. Run one of the following two cells to choose choose the solver.

In [ ]:
using SCS
using SetProg
factory = with_optimizer(SCS.Optimizer, verbose=0);

In [ ]:
using CSDP    # CSDP is less appropriate than SCS and Mosek because it does not natively support
using SetProg # SOC constraints so they need to be bridged to SDP constraints.
factory = with_optimizer(CSDP.Optimizer, printlevel=0); # SOC constraints are needed for the nth_root

In [ ]:
using MathOptInterfaceMosek
using SetProg
factory = with_optimizer(MosekOptimizer);

## John ellipsoid

The maximal volume ellispoid contained in a convex body is called its John ellipsoid.
The John ellipsoid for our square can be computed as follows.

In [ ]:
model = Model(factory);
@variable(model, john, Ellipsoid(dimension = 2))
cref = @constraint(model, john ⊆ p)
@objective(model, Max, nth_root(volume(john)))
@time JuMP.optimize!(model)
@show JuMP.termination_status(model)
@show JuMP.objective_value(model);

## Löwner ellipsoid

The minimal volume ellispoid contained in a convex body is called its Löwner ellipsoid.
The Löwner ellipsoid for our square can be computed as follows.

In [ ]:
model = Model(factory);
@variable(model, löwner, Ellipsoid(dimension = 2))
cref = @constraint(model, p ⊆ löwner)
@objective(model, Min, nth_root(volume(löwner)))
@time JuMP.optimize!(model)
@show JuMP.termination_status(model)
@show JuMP.objective_value(model);

We can visualize the Löwner and John ellispoids as follows.

In [ ]:
using Plots
plot(ratio=:equal)
plot!(JuMP.value(löwner))
plot!(p)
plot!(JuMP.value(john))

## Higher degree polynomials

Ellispoids are the sublevel sets of positive definite *quadratic* forms.
To allow for more sophisticated shapes, we could instead look for sublevel sets of *quartic* forms.
For this, we simply need to replace `Ellipsoid(dimension=2)` by `PolySet(degree=4, dimension=2)`.
Note that the volumes optimized are not exactly the volume anymore but provide a reasonable heuristic.

### Maximal volume quartic sublevel set contained in the square

In [ ]:
model = Model(factory);
@variable(model, quartic_in, PolySet(degree = 4, dimension = 2))
cref = @constraint(model, quartic_in ⊆ p)
@objective(model, Max, nth_root(volume(quartic_in)))
@time JuMP.optimize!(model)
@show JuMP.termination_status(model)
@show JuMP.objective_value(model);

### Minimal volume quartic sublevel set containing the square

In [ ]:
model = Model(factory);
@variable(model, quartic_out, PolySet(degree = 4, dimension = 2))
cref = @constraint(model, p ⊆ quartic_out)
@objective(model, Min, nth_root(volume(quartic_out)))
@time JuMP.optimize!(model)
@show JuMP.termination_status(model)
@show JuMP.objective_value(model);

We can visualize the quartic sublevel sets as follows.

In [ ]:
using Plots
plot(ratio=:equal)
plot!(JuMP.value(quartic_out))
plot!(p)
plot!(JuMP.value(quartic_in))